## 시세 가져오기

In [0]:
import requests
import pandas as pd
import os
from pandas import DataFrame
from bs4 import BeautifulSoup as bs

In [0]:

# n_line 라인의 type 기준의 시세 가져오기
def web_crawling(code, n_line, base):
    url = "https://fchart.stock.naver.com/sise.nhn?symbol="+code+"&timeframe="+base+"&count="+n_line+"&requestType=0"
    html = requests.get(url).text
    
    # item 태크로 찾기
    data_to_parse = bs(html)
    item_data = data_to_parse.find_all("item")
    
    if( len(item_data) == 0):
        return
    
    data_list = []

    # data 속성으로 찾기
    for i in item_data:
        data_list.append(i.get("data"))

    # delemiter로 slpit 하기
    for i in range(0,len(data_list)) :
        data_list[i] = data_list[i].split("|")
        
        
    data_df = pd.DataFrame(data_list)

    # 필요없는 column들은 제외해준다. 
    data_df = data_df[[ 0,4,5]]
    data_df['code'] = code


    # 컬럼명 바꿔준다. 
    data_df = data_df.rename(columns={0: 'DATE', 4: '종가', 5:'거래량'}) 
    
    return data_df

In [0]:
# 종목코드 리스트 가져오기
stock_list = pd.read_csv("stock_df.csv",  converters={'code': str})
code_list = stock_list['code'].tolist()

In [0]:
# price_df = pd.DataFrame()
price_df_monthly = pd.DataFrame()
for code in code_list:
#     price_df = price_df.append(web_crawling(str(code), "900", "day"))
    price_df_monthly = price_df_monthly.append(web_crawling(str(code), "60", "month"))
    

In [6]:
len(price_df_monthly)

43328

In [7]:
price_df_monthly.tail(3)

DATE   종가        거래량    code
57  20190531  616  121002970  003280
58  20190628  610   51962304  003280
59  20190724  568   10567035  003280

In [0]:
price_df_monthly.to_csv("price_df_monthly.csv")